# Support and Resistance Calculations

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
kotak_data = pd.read_csv('kotak.csv')

In [5]:
kotak_data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread'], dtype='object')

In [6]:
kotak_data.index

Index(['01-02-2020', '31-01-2020', '30-01-2020', '29-01-2020', '28-01-2020',
       '27-01-2020', '24-01-2020', '23-01-2020', '22-01-2020', '21-01-2020',
       ...
       '05-02-2018', '02-02-2018', '01-02-2018', '31-01-2018', '30-01-2018',
       '29-01-2018', '25-01-2018', '24-01-2018', '23-01-2018', '22-01-2018'],
      dtype='object', length=500)